In [1]:
import os

In [2]:
os.chdir("..")

In [3]:
os.getcwd()

'/home/markus/Documents/Projects/conservation_synthesis_github'

In [4]:
from sources.databases.internal_databases import SQLiteDB
from sources.frontend.app import get_journals
from sources.databases.journal_name_issn_database import JournalNameIssnDatabase

In [5]:
import pandas as pd

In [6]:
db = SQLiteDB()
db.initialise()
data = pd.DataFrame(db._get_all_data(0, 2000000))
db.terminate()

In [142]:
import re
import datetime

standard_date = re.compile(r"(\d{4})-(\d{1,2})-(\d{1,2})")
partial_date = re.compile(r"(\d{4})-(\d{1,2})")
dot_date = re.compile(r"(\d{4}).0")

def fix_date(date):
    if date is None:
        return None
    
    match =  standard_date.fullmatch(date.strip())
    if match is not None:
        return datetime.date(int(match.group(1)), int(match.group(2)), int(match.group(3))).isoformat()
    
    match = dot_date.fullmatch(date.strip())
    if match is not None:
        return datetime.date(int(match.group(1)), 1, 1).isoformat()
    
    match = partial_date.fullmatch(date.strip())
    if match is not None:
        return datetime.date(int(match.group(1)), 1, 1).isoformat()
    
    raise Exception("OH NO!")
    
def is_date(date):
    if date is None :
        return False
    val = standard_date.fullmatch(date.strip()) is not None 
    val = val or partial_date.fullmatch(date.strip()) is not None
    val = val or dot_date.fullmatch(date.strip()) is not None
    if val is False:
        print(date)
    return val

In [140]:
test = "2003.0"
fix_date(test)

'2003-01-01'

In [101]:
standard_date = re.compile(r"\d{4}.0")
standard_date.match()

<re.Match object; span=(0, 6), match='2003-4'>

In [111]:
def get_first(ls):
    return list(map(lambda p: p[0], filter(lambda p: not p[1], ls)))

In [121]:
return_val= data.apply(lambda row: (row[0], is_date(row[4])), axis=1)
data.set_index(keys=0, drop=True, inplace=True)
data.loc[get_first(return_val),:]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,,,,,,,,,,,,,,,,,,
1004743,ALEXANDER SMITH.,A W Menzies,10.1126/science.56.1450.409,None,"doctors with a kindliness, a tolerance and lar...",sem_scholar,EN,None,Science,56 1450,None,0036-8075,www.doi.org/10.1126/science.56.1450.409,2021-03-09,0,None,0,0.005855
1008169,THE MARINE BIOLOGICAL LABORATORY AT WOOD'S HOLL.,C. S. Bacon,10.1001/jama.1894.02421210009001d,None,"Medicine, which lays under contribution all th...",sem_scholar,EN,None,Science,9 229,None,0036-8075,www.doi.org/10.1001/jama.1894.02421210009001d,2021-03-09,0,None,0,0.004507
1032648,RESEARCH IN MEDICAL SCHOOLS.,F R Sabin,10.1126/science.65.1683.308,None,"In this well-received address, Sabin reflects ...",sem_scholar,EN,None,Science,65 1683,None,0036-8075,www.doi.org/10.1126/science.65.1683.308,2021-03-09,0,None,0,0.002540
1036052,WALCOTT ON THE CAMBRIAN FAUNAS.,Charles D. Walcott,10.1126/science.ns-9.226s.545,None,Improved enteric coated digestive enzyme-conta...,sem_scholar,EN,None,Science,9 226S,None,0036-8075,www.doi.org/10.1126/science.ns-9.226s.545,2021-03-09,0,None,0,0.004305
1044729,THE PROPOSED NATIONAL ARBORETUM AT WASHINGTON.,F V Coville,10.1126/science.62.1617.579,None,Nature. The proposed national arboretum at Was...,sem_scholar,EN,None,Science,62 1617,None,0036-8075,www.doi.org/10.1126/science.62.1617.579,2021-03-09,0,None,0,0.003466
1047088,STATISTICAL STUDY OF THE INFLUENZA EPIDEMIC.,E W Kopf,10.1001/jama.1919.02610080059028,None,To the Editor: —The American Public Health Ass...,sem_scholar,EN,None,Science,49 1262,None,0036-8075,www.doi.org/10.1001/jama.1919.02610080059028,2021-03-09,0,None,0,0.000994
1056114,PLAN OF THE BICENTENARY EXPEDITION TO THE NORT...,L Koch,10.1126/science.52.1354.552,None,"have thought much of relative values in life, ...",sem_scholar,EN,None,Science,52 1354,None,0036-8075,www.doi.org/10.1126/science.52.1354.552,2021-03-09,0,None,0,0.001833
1057271,A PRELIMINARY ATTEMPT TO TRANSMUTE LITHIUM.,Ralph W. G. Wyckoff,10.1126/science.55.1414.130,None,If an electron could be introduced into the \n...,sem_scholar,EN,None,Science,55 1414,None,0036-8075,www.doi.org/10.1126/science.55.1414.130,2021-03-09,0,None,0,0.000674
1158149,BASHFORD DEAN.,W K Gregory,10.1126/science.68.1774.635,None,the laws of biology. They do not have full fai...,sem_scholar,EN,None,Science,68 1774,None,0036-8075,www.doi.org/10.1126/science.68.1774.635,2021-03-09,0,None,0,0.002386


We can now fix stuff!

In [175]:
data.set_index(keys=0, drop=True, inplace=True)

In [144]:
data.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,,,,,,,,,,,,,,,,,,
1,Test,abc;bed,1234,2021-03-07,,testrepo,EN,None,None,None,None,None,None,None,b'0',None,0,NaN
118,In Memoriam Charles Henry Peterson,Hunter S.Lenihan;JonathanGrabowski;FiorenzaMic...,10.3354/meps13649,2021-2-4,None,crossref,EN,Inter-Research Science Center,None,None,None,0171-8630,https://doi.org/10.3354/meps13649,2021-03-07,b'0',NA,0,NaN
119,Diverse herbaceous cover crops promote vineyar...,Michael J. Samways;René Gaigher;James S. Pryke...,10.1016/j.agee.2020.107222,2021-2,Abstract Agricultural intensification threaten...,openaire,EN,Elsevier BV,"Agriculture, Ecosystems & Environment",307,None,0171-8630,https://doi.org/10.1016/j.agee.2020.107222,2021-03-07,b'0',NA,1,0.981580
120,Sugar feeding by parasitoids inside and around...,Miriam Kishinevsky;Tamar Keasar,10.1016/j.agee.2020.107229,2021-2,Abstract Feeding on sugar-rich resources drama...,openaire,EN,Elsevier BV,"Agriculture, Ecosystems & Environment",307,None,0171-8630,https://doi.org/10.1016/j.agee.2020.107229,2021-03-07,b'0',NA,1,0.580991
121,Environmental soil quality and vegetable safet...,Chao Ma;Yusef Kianpoor Kalkhajeh;Wenyou Hu;Bia...,10.1016/j.agee.2020.107230,2021-2,Abstract Rapid expansion of greenhouse vegetab...,openaire,EN,Elsevier BV,"Agriculture, Ecosystems & Environment",307,None,0171-8630,https://doi.org/10.1016/j.agee.2020.107230,2021-03-07,b'0',NA,0,0.052421


In [ ]:
col_name = "checked"

In [176]:
db = SQLiteDB()
db.initialise()
for ind, row in data.iterrows():
    db._update_col(ind, "checked", 0)
    db._update_col(ind, "classified", None)
db.terminate()



In [7]:
pos_data = pd.read_json("data/positive_samples/positive_samples_v2_170221_corrected_journals_removed_tags.pd.json")


In [10]:
cnt=0
db = SQLiteDB()
db.initialise()
for ind, row in pos_data.iterrows():
    if row["doi"] is not None:
        db._update_col_doi_based(row["doi"], "checked", 1)
db.terminate()